In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  9% |  0% |


In [3]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
import os

In [29]:
os.chdir('input/data')
os.getcwd()

'/opt/ml/input/data'

In [30]:
!find . -regex ".*\.\_[a-zA-Z0-9._]+" -delete

In [4]:
random_seed = 12
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [5]:
train_dir_path = '/opt/ml/input/data/train/'
train_image_path = '/opt/ml/input/data/train/images/'

dt_train = pd.read_csv(train_dir_path+'train.csv')
dt_train

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [6]:
whole_image_path = []
whole_target_label = []

for path in dt_train['path']:
    for file_name in [i for i in os.listdir(train_image_path+path) if './' not in i]:
        whole_image_path.append(train_image_path+path+'/'+file_name)
        whole_target_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))

In [7]:
def onehot_enc(x):
    def gender(i):
        if i == 'male':
            return 0
        elif i == 'female':
            return 3
    def age(j):
        j = int(j)
        if j < 30:
            return 0
        elif j >= 30 and j < 60:
            return 1
        elif j >= 60:
            return 2
    def mask(k):
        if k == 'normal':
            return 12
        elif 'incorrect' in k:
            return 6
        else:
            return 0
    return gender(x[0]) + age(x[1]) + mask(x[2])

In [8]:
sr_data = pd.Series(whole_image_path)
sr_label = pd.Series(whole_target_label)

In [9]:
class Dataset_Mask(Dataset):
    def __init__(self, encoding=True, midcrop=True, transform=None):
        self.encoding = encoding
        self.midcrop = midcrop
        self.data = sr_data
        self.label = sr_label
        self.transform = transform
        
        if encoding:
            self.label = self.label.apply(onehot_enc)
        
    def __len__(self):
        return len(sr_data)
    
    def __getitem__(self, idx):
        X = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        y = self.label[idx]
        
        if self.midcrop:
            X = X[80:380, 42:342]
        
        if self.transform:
            return self.transform(X), y
        return X, y

In [10]:
dataset_mask = Dataset_Mask(transform = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Resize(224),
                                transforms.ToTensor()
                            ]))

In [11]:
train_size = int(len(dataset_mask) * 0.8)
val_size = int(len(dataset_mask) * 0.2)

In [12]:
mask_train_set, mask_val_set = torch.utils.data.random_split(dataset_mask, [train_size, val_size])
print(f'training data size : {len(mask_train_set)}')
print(f'validation data size : {len(mask_val_set)}')

training data size : 15120
validation data size : 3780


In [13]:
batch_size = 256
train_dataloader_mask = DataLoader(dataset = mask_train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_dataloader_mask = DataLoader(dataset = mask_val_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [14]:
import timm
class SwinNet(nn.Module):
    def __init__(self, model_name='swin_small_patch4_window7_224', out_features=18,
                inp_channels=3, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained,
                                      in_chans=inp_channels)
        for param in self.model.parameters():
            param.requires_grad = False
        n_features = self.model.head.in_features
        self.model.head = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, out_features)
        )
        
    def forward(self, x):
        return self.model(x)

In [15]:
from timm.loss import LabelSmoothingCrossEntropy
from torch.optim.lr_scheduler import CosineAnnealingLR

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

model = SwinNet()
model = model.to(device)

LEARNING_RATE = 1e-4
NUM_EPOCH = 100

criterion = LabelSmoothingCrossEntropy().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE,
                            weight_decay=1e-6,
                            amsgrad=False)
scheduler = CosineAnnealingLR(optimizer, 10,
                             eta_min=1e-6,
                             last_epoch=-1)

using cuda:0


In [16]:
best_val_acc = 0
best_val_loss = np.inf
patience = 5
cur_count = 0

for epoch in range(NUM_EPOCH):
    model.train()
    loss_value = 0
    matches = 0
    for train_batch in train_dataloader_mask:
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        
        if epoch % 10 == 0:
            torch.save(model, '/opt/ml/checkpoint/swint_small_patch4_window7_224/checkpoint_ep_%d.pt'% epoch)
        
        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        
        train_loss = loss_value / batch_size
        train_acc = matches / batch_size
        print(f"epoch[{epoch}/{NUM_EPOCH}] training loss {train_loss:.5f}, training accuracy {train_acc:.5f}")
        
        loss_value = 0
        matches = 0
        
    with torch.no_grad():
        model.eval()
        val_loss_items = []
        val_acc_items = []
        for val_batch in val_dataloader_mask:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            
            loss_item = criterion(outs, labels).item()
            acc_item = (labels==preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            
        val_loss = np.sum(val_loss_items) / len(val_dataloader_mask)
        val_acc = np.sum(val_acc_items) / len(mask_val_set)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            cur_count = 0
            torch.save(model, '/opt/ml/checkpoint/swint_small_patch4_window7_224/checkpoint_best.pt')
        else:
            cur_count += 1
            if cur_count >= patience:
                print("Early Stopping!")
                break
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            
        print(f"[val] acc : {val_acc:.5f}, loss : {val_loss:.5f}")
        print(f"best acc : {best_val_acc:.5f}, best loss : {best_val_loss:.5f}")

epoch[0/100] training loss 0.01123, training accuracy 0.09375
epoch[0/100] training loss 0.01109, training accuracy 0.13672
epoch[0/100] training loss 0.01099, training accuracy 0.18359
epoch[0/100] training loss 0.01088, training accuracy 0.22266
epoch[0/100] training loss 0.01078, training accuracy 0.25000
epoch[0/100] training loss 0.01080, training accuracy 0.19141
epoch[0/100] training loss 0.01069, training accuracy 0.25391
epoch[0/100] training loss 0.01075, training accuracy 0.21484
epoch[0/100] training loss 0.01073, training accuracy 0.23047
epoch[0/100] training loss 0.01071, training accuracy 0.16406
epoch[0/100] training loss 0.01068, training accuracy 0.24609
epoch[0/100] training loss 0.01065, training accuracy 0.23047
epoch[0/100] training loss 0.01062, training accuracy 0.21094
epoch[0/100] training loss 0.01068, training accuracy 0.25000
epoch[0/100] training loss 0.01071, training accuracy 0.22656
epoch[0/100] training loss 0.01060, training accuracy 0.23438
epoch[0/

In [17]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir_path = '/opt/ml/input/data/eval/'
test_image_path = '/opt/ml/input/data/eval/images/'

best_model = torch.load('/opt/ml/checkpoint/swint_small_patch4_window7_224/checkpoint_best.pt')
submission = pd.read_csv(test_dir_path+'info.csv')
submission.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0


In [18]:
image_paths = [os.path.join(test_image_path, img_id) for img_id in submission.ImageID]
test_image = pd.Series(image_paths)

In [19]:
class Test_Dataset(Dataset):
    def __init__(self, midcrop=True, transform=None):
        self.midcrop = midcrop
        self.data = test_image
        self.transform = transform
        
    def __len__(self):
        return len(test_image)
    
    def __getitem__(self, idx):
        img = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        
        if self.midcrop:
            img = img[64:448]
            
        if self.transform:
            img = self.transform(img)
            
        return img

In [20]:
dataset = Test_Dataset(transform = transforms.Compose([
                            transforms.ToPILImage(),
                            transforms.Resize(224),
                            transforms.ToTensor()
                        ]))

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = best_model.to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir_path, 'submission_swin_transformer_t_100.csv'), index=False)
print('test inference is done!')

test inference is done!


In [ ]:
/opt/ml/checkpoint/swin_transfrmer_t_2/